<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fbigdatamagazine.es%2Fwp-content%2Fuploads%2F2023%2F02%2FFOTO-OK-BDM-DIG-DATA-MAGAZINE.jpg&f=1&nofb=1&ipt=4061921fa0da07483f83edb036d31f25545b2cae889c7eeefebd576f6e0fe5f4" style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

# Introducción

En este tutorial aprenderás cómo utilizar **pipelines** para mantener tu código de modelado limpio y organizado.

## ¿Qué es un Pipeline?

Un **pipeline** (o flujo de procesamiento) es una forma sencilla de agrupar los pasos de preprocesamiento y modelado de datos, de manera que puedas tratar todo el conjunto como si fuera una única operación.

Muchos científicos de datos construyen modelos sin usar pipelines, pero estos ofrecen beneficios importantes, entre los cuales se encuentran:

1. **Código más limpio:** Gestionar los datos en cada paso del preprocesamiento puede volverse complicado. Con un pipeline, no necesitas llevar el control manual del conjunto de entrenamiento y validación en cada paso.
2. **Menos errores:** Se reducen las posibilidades de aplicar mal un paso o de olvidar una etapa de preprocesamiento.
3. **Facilidad para pasar a producción:** Migrar un modelo desde un prototipo a un entorno de producción puede ser sorprendentemente difícil. Aunque no abordaremos todos los aspectos relacionados aquí, los pipelines facilitan este proceso.
4. **Más opciones para la validación del modelo:** Verás un ejemplo de esto en el siguiente tutorial, el cual cubre la validación cruzada.

# Ejemplo

Como en el tutorial anterior, trabajaremos con el [conjunto de datos de viviendas de Melbourne](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home).

No nos enfocaremos en la etapa de carga de datos. En su lugar, puedes imaginar que ya cuentas con los siguientes subconjuntos preparados:

- `X_train`
- `X_valid`
- `y_train`
- `y_valid`


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Leer los datos
data = pd.read_csv('/content/melb_data.csv')

# Separar la variable objetivo de los predictores
y = data.Price
X = data.drop(['Price'], axis=1)

# Dividir los datos en subconjuntos de entrenamiento y validación
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinalidad" se refiere al número de valores únicos en una columna
# Seleccionar columnas categóricas con cardinalidad relativamente baja (elección práctica pero arbitraria)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and
                        X_train_full[cname].dtype == "object"]

# Seleccionar columnas numéricas
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Conservar únicamente las columnas seleccionadas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()


Echamos un vistazo a los datos de entrenamiento con el método `head()` Observa que los datos contienen tanto **variables categóricas** como **columnas con valores faltantes**. ¡Con un **pipeline**, es fácil tratar con ambos casos!


In [3]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


Construimos el pipeline completo en tres pasos.

### Paso 1: Definir los pasos de preprocesamiento

De forma similar a cómo una pipeline agrupa pasos de preprocesamiento y modelado, usamos la clase `ColumnTransformer` para agrupar distintos pasos de preprocesamiento. El siguiente código:

- Imputa los valores faltantes en los datos **_numéricos_**, y  
- Imputa los valores faltantes y aplica codificación one-hot a los datos **_categóricos_**.


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocesamiento para datos numéricos
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocesamiento para datos categóricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # Codificación one-hot ignorando categorías desconocidas
])

# Agrupar el preprocesamiento de datos numéricos y categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),  # Transformación de columnas numéricas
        ('cat', categorical_transformer, categorical_cols)  # Transformación de columnas categóricas
    ])

### Paso 2: Definir el Modelo

A continuación, definimos un modelo de bosque aleatorio utilizando la conocida clase [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html).



In [5]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

### Paso 3: Crear y Evaluar la Pipeline

Finalmente, utilizamos la clase [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) para definir una pipeline que agrupa los pasos de preprocesamiento y modelado. Hay algunos aspectos importantes a destacar:

- Con el pipeline, se realiza el preprocesamiento de los datos de entrenamiento y el ajuste del modelo en una sola línea de código.  
  (_En contraste, sin una pipeline, tendríamos que realizar la imputación, codificación one-hot y entrenamiento del modelo en pasos separados. Esto se vuelve especialmente complicado cuando trabajamos con variables tanto numéricas como categóricas._)

- Con el pipeline, simplemente se pasan las características sin procesar en `X_valid` al comando `predict()`, y la pipeline se encarga automáticamente de preprocesarlas antes de generar las predicciones.  
  (_Sin embargo, sin una pipeline, debemos recordar preprocesar manualmente los datos de validación antes de hacer predicciones._)


In [6]:
from sklearn.metrics import mean_absolute_error

# Agrupar el preprocesamiento y el modelo en una pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocesamiento de los datos de entrenamiento y ajuste del modelo
my_pipeline.fit(X_train, y_train)

# Preprocesamiento de los datos de validación y generación de predicciones
preds = my_pipeline.predict(X_valid)

# Evaluar el modelo
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)


MAE: 160679.18917034855


# Conclusión

Las *pipelines* (o flujo de procesamiento) son valiosas para organizar el código de *machine learning* y evitar errores, especialmente en flujos de trabajo que requieren un preprocesamiento de datos sofisticado. Permiten encapsular todo el proceso —desde la limpieza de datos hasta el entrenamiento del modelo— en una única estructura coherente y reutilizable.
